In [39]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

pd.set_option('display.max_rows', 100)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [44]:
file_path = '/home/carolus/Documents/school/green_ia/data/05_data/05_valid_01.jsonl'
num_samples = 500

In [45]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df

In [46]:
df = sample_jsonl_file(file_path, num_samples)

In [47]:
df.tail(60)

,groups,packaging,name,ecoscore_tags,ecoscore_score,countries,ingredients,categories,labels_note
440,0.444444,empty,ultra-pasteurized half & half,b,64,1.000000,"milk, dairy, cream, contains-less-than-1-of, e...","dairies, creams",0.000000
441,0.000000,empty,arataki manuka honey,b,72,0.311475,empty,"breakfasts, spreads, sweet-spreads, bee-produc...",0.000000
442,0.222222,"plastique, carton, frais",bœuf bourguignon mitonné au vin de bourgogne,e,16,0.000000,"beef-meat, animal, meat, beef, water, red-wine...","meats-and-their-products, meals, meals-with-me...",0.555556
443,0.777778,empty,soya drink unsweetened,b,69,0.968085,"water, soya-bean, vegetable, legume, pulse, so...","beverages-and-beverages-preparations, plant-ba...",0.333333
444,0.444444,plástico,inline deligurt yogurt líquido arándano,d,23,0.311475,empty,"beverages, dairies, fermented-foods, fermented...",0.000000
445,0.555556,"pet 1 jar, pp 5 lid",chunky peanut butter,c,41,0.813830,"roasted-peanuts, nut, peanut, peanut-oil, oil-...","plant-based-foods-and-beverages, plant-based-f...",0.111111
446,0.000000,empty,chocolat au lait noisette du piemont,d,33,0.311475,empty,"snacks, sweet-snacks, cocoa-and-its-products, ...",0.444444
447,0.666667,empty,cadbury micro mini eggs,d,30,0.158470,empty,"farming-products, eggs",0.000000
448,0.555556,empty,seminte 300g negre fara sare,d,29,0.311475,empty,"plant-based-foods-and-beverages, plant-based-f...",0.000000
449,0.555556,empty,gourmet granola,d,25,0.158470,"wholemeal-spelt-flour, flour, cereal-flour, wh...","plant-based-foods-and-beverages, plant-based-f...",0.111111


In [25]:
uniques_groups = df['groups'].unique()
uniques_groups_list = list(uniques_groups)
for value in uniques_groups_list:
    print(value)

0.4444444444
0.6666666667
1.0
0.2222222222
0.3333333333
0.5555555556
0.7777777778
0.1111111111
0.0


In [26]:
uniques_name = df['name'].unique()
uniques_name_list = list(uniques_name)
for value in uniques_name_list:
    print(value)

crujientes de bacalao sabor curry
confiture noix verte
liesa varske
schweizer poulet zum gekochtessen
schaschlik
froxitos
latte intero
багет зерновой
hähnchen innefilet
4 quesos rallado
rhythm superfoods, kool ranch kale chips
fondue de quesos suizos
chocolate mini cupcakes
yogurt bio myrtille
kraft extra sharp cheddar 24oz
foie gras de cabard entier du dud ouest
besciamella
orelha suína salgada sem osso prieto 400g
mortadella gran selezionr
kibbeh
wald honig
pipoca doce
pasta di mandorle
tomate cube 1kg
lait choco
bob's red mill, organic whole grain stone ground spelt flour
spicy sauce au piments français
maionese al tartufo
energy refreshed lifestyle beverage
organic brown rice lentil patties
melange bio energisant
rainbow peppercorn wild alaskan salmon jerky, rainbow peppercorn
mélange jeunes pousses
buffalo sauce
bodyvit vitamin drink
pizzasaus tomat - chiliflakes krydret med sort pepper
car mix noix grille sel
none
pompelmo in rosa
schinken räucher
panettone amarena e cioccolato
p

In [27]:
uniques_countries = df['countries'].unique()
uniques_countries_list = list(uniques_countries)
for value in uniques_countries_list:
    print(value)

0.8707865169
0.0
0.9945355191
0.3114754098
0.3333333333
0.8743169399
0.4316939891
0.7446808511
1.0
0.5762711864
0.9016393443
0.9572192513
0.4124293785
0.1147540984
0.1117021277
0.420212766
0.5355191257
0.2960893855
0.6885245902
0.0765027322
0.2864864865
0.2881355932
0.7486338798
0.2544378698
0.9734042553
0.0558659218
0.9940828402
0.218579235
0.0327868852
0.6557377049
0.1584699454
0.7650273224
0.8715083799
0.2786885246
0.3031914894
0.9890710383
0.8432432432
0.3107344633
0.8639053254
0.3152173913
0.7431693989
0.4426229508
0.4207650273
0.8510638298
0.7595628415
0.2131147541
0.3149171271
0.4078212291
0.6536312849
0.6229508197
0.0167597765
0.2945205479
0.2857142857
0.8700564972
0.8961748634
0.3442622951
0.8994413408
0.7393617021
0.3184357542
0.2847682119
0.737704918
0.2834224599
0.5901639344
0.4033149171
0.8776595745
0.868852459
0.320754717
0.7514124294
0.1818181818
0.4301675978
0.4189944134
0.9625
0.9943502825
0.8684210526
0.0756756757
0.4308510638
0.3244680851
0.6382978723
0.956284153
0.1

In [28]:
uniques_packaging = df['packaging'].unique()
uniques_packaging_list = list(uniques_packaging)
for value in uniques_packaging_list:
    print(value)

plástico
empty
ldpe - polyethylen niedriger dichte
empaque de carton
plastique,frais,réfrigéré
kunststoff, tablettverpackung
sachet plastique
plastique, carton
plastic
backing
пластик
kunststoff
flacon plastique
etui en carton,pot en plastique
plastique
plastikowe
papier, surgelé
metal, can, 40 fe
boîte, conserve, boite métal
métal, verre, bocal, couvercle
lata, pasteurizado
plastique, barquette
verre
frais
glass-bottle
barquette plastique
pot
plástico, 07 other
paper carton
cardboard
plastique, frais, sous-vide
pet-pot, ldpe-wrapper
plastique, bouchon de bouteille, brique, carton, bouchon de bouteille de vin
glass,bottle
bag, mixed plastic film-packet
pet-bottle, pp-lid
papier
plastique, frais, barquette, barquette en plastique, jetez le barquette avec les déchets ménagers, point vert
plastique, sachet, carton
métal, plastique, recycler, verre, jeter, film, bocal, couvercle, bocal en verre, couvercle en métal, film en plastique
plastique, pp05
vidrio, bote, pasteurizado
sachet en allu

In [29]:
uniques_ecoscore_tags = df['ecoscore_tags'].unique()
uniques_ecoscore_tags_list = list(uniques_ecoscore_tags)
for value in uniques_ecoscore_tags_list:
    print(value)

empty
c
d
b
a
not-applicable
e


In [30]:
uniques_ecoscore_score = df['ecoscore_score'].unique()
uniques_ecoscore_score_list = list(uniques_ecoscore_score)
for value in uniques_ecoscore_score_list:
    print(value)

49.0
52.0
33.0
21.0
35.0
74.0
67.0
100.0
13.0
39.0
44.0
12.0
15.0
51.0
62.0
56.0
18.0
68.0
78.0
31.0
46.0
79.0
64.0
77.0
59.0
54.0
72.0
66.0
22.0
80.0
10.0
25.0
97.0
40.0
57.0
55.0
45.0
34.0
28.0
47.0
27.0
58.0
71.0
7.0
0.0
36.0
43.0
17.0
75.0
30.0
24.0
76.0
48.0
65.0
19.0
8.0
60.0
88.0
70.0


In [31]:
uniques_ingredients = df['ingredients'].unique()
uniques_ingredients_list = list(uniques_ingredients)
for value in uniques_ingredients_list:
    print(value)

cod, fish, white-fish, high-oleic-sunflower-oil, oil-and-fat, vegetable-oil-and-fat, vegetable-oil, sunflower-oil, curry, condiment, spice, e621
water
empty
whole-chicken, poultry, chicken
kale, vegetable, brassica, cabbage, sunflower-seed, plant, seed, sunflower, tahini, sesame, sesame-paste, carrot, root-vegetable, taproot-vegetable, cane-sugar, added-sugar, disaccharide, sugar, cider-vinegar, vinegar, onion, onion-family-vegetable, sea-salt, salt, lemon-juice, fruit, juice, fruit-juice, dill-weed, garlic-white-pepper, sesame-seeds
white-wine, alcohol, wine, potato-starch, starch, e410, iodised-salt, salt, spice, condiment, pasteurised-cow-s-milk, dairy, milk, pasteurised-milk, cow-s-milk, enzyme, lactic-ferments, ferment, microbial-culture
cooked-beef-meat, animal, meat, beef, beef-meat, onion, vegetable, root-vegetable, onion-family-vegetable, water, cracked-bulgur-wheat, flour, paprika, condiment, spice, salt, black-pepper, seed, pepper, allspice, fruit-vegetable, chili-pepper, cu

In [32]:
uniques_categories = df['categories'].unique()
uniques_categories_list = list(uniques_categories)
for value in uniques_categories_list:
    print(value)

empty
meats-and-their-products, frozen-foods, meats, chicken-and-its-products, poultries, frozen-meats, chickens, whole-chickens
dairies, milks, whole-milks
meats-and-their-products, meats, chicken-and-its-products, poultries, chickens
dairies, fermented-foods, fermented-milk-products, cheeses, grated-cheese
dairies, fermented-foods, fermented-milk-products, cheeses, spreads, salted-spreads, cheese-spreads
dairies, fermented-foods, fermented-milk-products, desserts, dairy-desserts, fermented-dairy-desserts, yogurts
meats-and-their-products, beef-and-its-products, frozen-foods, meals, frozen-ready-made-meals, kibbeh
plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetables-based-foods, vegetables-based-foods, vegetables, tomatoes-and-their-products, tomatoes
plant-based-foods-and-beverages, plant-based-foods, cereals-and-potatoes, cereals-and-their-products
beverages, carbonated-drinks, sodas
snacks
plant-based-foods-and-beverages, plant-based-foods, fruits-and-vegetable

In [33]:
uniques_labels_note = df['labels_note'].unique()
uniques_labels_note_list = list(uniques_labels_note)
for value in uniques_labels_note_list:
    print(value)

0.1111111111
0.2222222222
0.0
0.3333333333
0.4444444444
0.5555555556
1.0
0.6666666667
0.7777777778
0.8888888889


In [34]:
df.describe()

,groups,ecoscore_score,countries,labels_note
count,500.000000,500.000000,500.000000,500.000000
mean,0.434000,49.256000,0.527731,0.072000
std,0.201836,11.786786,0.336454,0.142612
min,0.000000,0.000000,0.000000,0.000000
25%,0.444444,49.000000,0.311475,0.000000
50%,0.444444,49.000000,0.333333,0.000000
75%,0.444444,49.000000,0.874317,0.111111
max,1.000000,100.000000,1.000000,1.000000


In [35]:
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

def create_treemap_for_column(df, column_name):
    if column_name not in df.columns:
        print(f"warning, column: '{column_name}'does not exists")
        return
    data = df[column_name].value_counts().reset_index()
    data.columns = [column_name, 'count']
    fig = go.Figure(go.Treemap(
        labels=data[column_name],
        parents=[""] * len(data),
        values=data['count'],
        textinfo='label+value',  
    ))
    fig.update_layout(
        title=f'treemap column: {column_name}',
        paper_bgcolor='black',  
        plot_bgcolor='grey',   
        font_color='white'      
    )
    fig.show()

In [36]:
colonnes_a_travailler = ['ecoscore_tags', 'ecoscore_score', 'countries', 'labels_note']  
df = load_jsonl_to_dataframe(file_path)
for column in colonnes_a_travailler:
    create_treemap_for_column(df, column)

In [37]:
def count_unique_values(jsonl_file_path, colonnes_a_travailler):
    data = []
    with open(jsonl_file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    unique_counts = {}
    for column in colonnes_a_travailler:
        if column in df.columns:
            unique_count = df[column].nunique()
            if pd.api.types.is_numeric_dtype(df[column]):
                min_val = df[column].min()
                max_val = df[column].max()
                unique_counts[column] = {
                    'unique_count': unique_count,
                    'min': min_val,
                    'max': max_val
                }
            else:
                unique_counts[column] = {
                    'unique_count': unique_count
                }
        else:
            unique_counts[column] = 'Column not found in data'
    return unique_counts

In [38]:
unique_values = count_unique_values(file_path, colonnes_a_travailler)
print(unique_values)

{'ecoscore_tags': {'unique_count': 7}, 'ecoscore_score': {'unique_count': 102, 'min': 0.0, 'max': 100.0}, 'countries': {'unique_count': 932, 'min': 0.0, 'max': 1.0}, 'labels_note': {'unique_count': 10, 'min': 0.0, 'max': 1.0}}
